In [30]:
import os
import sys
import random
import warnings

import numpy as np
import pandas as pd
import time

import matplotlib.pyplot as plt
import io
import pickle
import json

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.preprocessing.sequence import pad_sequences
from ast import literal_eval

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

#import torchvision.datasets as dset
#import torchvision.transforms as T
import torch.nn.functional as F
from basic import *
from rnn_lstm import *

In [2]:
# For this cell used same code from PyTorch notebook in assignment 2 of Stanford's CS231n Spring 2018 offering
preprocessData = False # To preprocess data set this to True
USE_GPU = False
dtype = torch.float32
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    dtype = torch.float32

# Constant to control how frequently we print train loss
print_every = 1

print('using device:', device)

using device: cpu


In [3]:
# Next two cells, code belongs to [1]. Minor changes made to accomodate to our use 
# (Using PyTorch instead of Keras/tensorflow)
STROKE_COUNT = 196
IMG_WIDTH = 28
IMG_HEIGHT = 28
IMG_CHANNELS = 1
PATH = './'
epsilon = 1e-12 #For numerical stability

warnings.filterwarnings('ignore', category=UserWarning, module='skimage')
seed = 1
random.seed = seed
np.random.seed = seed

In [ ]:
def stack_it(raw_strokes):
    """preprocess the string and make
    a standard Nx3 stroke vector"""

    stroke_vec = literal_eval(raw_strokes)# string->list
    #print('stroke_vec: ', stroke_vec)
    # unwrap the list
    in_strokes = [(xi,yi,i)
    for i,(x,y) in enumerate(stroke_vec)
    for xi,yi in zip(x,y)]
    c_strokes = np.stack(in_strokes)
    # replace stroke id with 1 for continue, 2 for new
    c_strokes[:,2] = [1]+np.diff(c_strokes[:,2]).tolist()
    c_strokes[:,2] += 1 # since 0 is no stroke
    # pad the strokes with zeros
    return pad_sequences(c_strokes.swapaxes(0, 1),
                         maxlen=STROKE_COUNT,
                         padding='post').swapaxes(0, 1)

In [ ]:
def CreateLabels(data):
    # Key: col 1: airplane, col 2: campfire, col 3: key, col 4: moon, col 5: palm tree
    #print('data: ', data)
    m = data.shape[0]
    print('m: ', m)
    labels = np.zeros((m, 5))
    for row in range(m):
        if row == 0:
            pass
        if data[row] == 'airplane':
            labels[row, 0] = 1
        elif data[row] == 'campfire':
            labels[row, 1] = 1
        elif data[row] == 'key':
            labels[row, 2] = 1
        elif data[row] == 'moon':
            labels[row, 3] = 1
        else:
            labels[row, 4] = 1
    print('labels: ', labels)
    return labels

In [ ]:
def Strokes(data):
    """
    Function looks at the dataframe and returns matrix of description
    condensed encodings for all samples (each sample has variable length vectors
        of indices corresponding to vocab indices of words that appear in each
        description)
    Arguments:
    df -- dataframe of data
    Returns:
    X -- product descriptions
    """    
    m = data.shape[0]
    #print('data: ', data)

    # Key: col 1: airplane, col 2: campfire, col 3: key, col 4: moon, col 5: palm tree
    #print('data: ', data)

    temp = []
    for i in range(data.shape[0]):
        temp.append(stack_it(data[i, 0]))
    X = np.asarray(temp)

    #Y= np.array(Y)
    X = np.asarray(X)
    print('X: ', X)
    print('X shape: ', X.shape)
    return X

In [4]:
# Load train data
trainCSV = "./train.csv"
trainDF = pd.read_csv(trainCSV, header = 0)
trainDF = trainDF.values
X_strokes = Strokes(trainDF)
X_train = np.swapaxes(X_strokes, 1, 2)

Y_t = trainDF[:, 2]
Y_labels = CreateLabels(Y_t)
Y_train = np.zeros((Y_labels.shape[0]))
# Pytorch needs indices
for i, row in enumerate(Y_labels):
    Y_train[i] = np.argmax(row)

print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))


X_train shape: (364984, 1, 28, 28)
Y_train shape: (364984,)
X_dev shape: (121661, 1, 28, 28)
Y_dev shape: (121661,)


In [ ]:
# Load dev data
devCSV = "./dev.csv"
devDF = pd.read_csv(devCSV, header = 0)
devDF = devDF.values
X_strokes = Strokes(devDF)
X_dev = np.swapaxes(X_strokes, 1, 2)

Y_d = devDF[:, 2]
Y_labels = CreateLabels(Y_d)
Y_dev = np.zeros((Y_labels.shape[0]))
# Pytorch needs indices
for i, row in enumerate(Y_labels):
    Y_dev[i] = np.argmax(row)

print ("X_dev shape: " + str(X_dev.shape))
print ("Y_dev shape: " + str(Y_dev.shape))

In [5]:
def showVisualComparisons(X, y, ex):
    plt.imshow(np.uint8(np.reshape(X[ex, :], (28, 28))))
    plt.show()
    print(Y_train[:, ex])

In [31]:
def trainModel(model, x_train, y_train, optimizer, epochs = 1, mini_batch_size = 64, noVal = False):
#     model = model.to(device=device)  # move the model parameters to CPU/GPU
    T = 0
    num_batches = int(len(x_train)/mini_batch_size)
    num_remaining = len(x_train) - num_batches * mini_batch_size
    loss_history = []
    epsilon = 0.0
    
    for e in range(epochs):
        correct = 0
        for t in range(num_batches):
            rand_indices = np.random.choice(len(x_train), mini_batch_size)
            x = torch.from_numpy(x_train[rand_indices, :, :, :])
            y = torch.from_numpy(y_train[rand_indices])
            model.train()  # put model to training mode
#             x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
#             y = y.to(device=device, dtype=dtype)
            y = y.type(torch.LongTensor)

            preds = model(x)
            preds = torch.squeeze(preds)
#             print(preds.size())
            _, predicted = torch.max(preds.data, 1)
            
            for i in range(len(predicted)):
                if predicted[i] == y[i]:
                    correct += 1
            
            loss = F.cross_entropy(preds, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if T % print_every == 0:
                currLoss = loss.item()
                loss_history.append(currLoss)
#                 print('Epoch %d, Iteration %d, loss = %.4f' % (e, t, currLoss))
            if (num_remaining <= 0 and t == (num_batches -1)):
#                 perf = calculatePerformance(x_train, y_train, model)
                perf = (correct / (float(mini_batch_size)))
                print('Train performance at epoch %d is %.4f' % (e, perf))
                if (noVal == False):
#                     perf = calculatePerformance(X_val, Y_val, model)
                    print('Val performance at epoch %d is %.4f' % (e, perf))
            T +=1
        if num_remaining > 0:
            rand_indices = np.random.choice(len(x_train), num_remaining)

            x = torch.from_numpy(x_train[rand_indices, :, :, :])
            y = torch.from_numpy(y_train[rand_indices])
            model.train()  # put model to training mode
#             x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
#             y = y.to(device=device, dtype=dtype)
            y = y.type(torch.LongTensor)
            


            preds = model(x)
            preds = torch.squeeze(preds)
            print(preds.size())
            
            _, predicted = torch.max(preds.data, 1)
            #values, indices = torch.max(preds, 1)
            
            for i in range(len(predicted)):
                if predicted[i] == y[i]:
                    correct += 1
                    
            loss = F.cross_entropy(preds, y)
            #loss(preds, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()
            if T % print_every == 0:
                currLoss = loss.item()
                loss_history.append(currLoss)
#                 print('Epoch %d, Iteration %d, loss = %.4f' % (e, num_batches, currLoss))
#             perf = (correct/(float(len(x_train))))
#             print('Train performance at epoch %d is %.4f' % (e, perf))
            if (noVal == False):
#                 perf = (correct/(float(len(x_train))))

                print('Val performance at epoch %d is %.4f' % (e, perf))
            T +=1
        perf = (correct / len(x_train))
        print(loss_history[-1])
        print('Train performance at epoch %d is %.4f' % (e, perf))
        if loss_history[-1] <= epsilon:
            break
    return perf, loss_history

In [49]:
# Overfitting data first
bestPerf = -1
lossHistory = None
lossHistories = {}
print_every = 1
bestModel = None
bestLoss = 10000
lrUsed = 0
half_X_train = X_train[0:len(X_train) // 2, :, :, :]
half_Y_train = Y_train[0:len(Y_train) // 2]
x_train = X_train[0:1000, :, :, :]
y_train = Y_train[0:1000]
lrs = []
# lrs.append(.002147418314081924) # best basic 
# lrs.append(0.0003513721088531244) #found another good basic
lrs.append(8.421693197665337e-05) # best conv
# for i in range(3):
#     lrs.append(5*np.random.rand()*1e-4)
# lrs = [1e-7,1e-6,1e-5,1e-4,1e-3]
for lr in lrs:
    print('Trying out learning rate of ', lr)
    model = CNNNet()
    optimizer = optim.Adam(model.parameters(), lr = lr)
    print(np.shape(x_train))
    print(np.shape(y_train))
    modelPerf = trainModel(model, x_train, y_train, optimizer, epochs = 50, noVal = True)
    lossHistories[str(lr)] = modelPerf[1]
    if modelPerf[1][len(modelPerf[1])-1] < bestLoss:
        bestLoss = modelPerf[1][len(modelPerf[1])-1]
        bestPerf = modelPerf[0]
        lossHistory = modelPerf[1]
        bestModel = model
        lrUsed = lr

Trying out learning rate of  8.421693197665337e-05
(1000, 1, 28, 28)
(1000,)
torch.Size([40, 5])
509.18194580078125
Train performance at epoch 0 is 0.2400
torch.Size([40, 5])
24.599872589111328
Train performance at epoch 1 is 0.5430
torch.Size([40, 5])
10.25886344909668
Train performance at epoch 2 is 0.7680
torch.Size([40, 5])
5.457827091217041
Train performance at epoch 3 is 0.8810
torch.Size([40, 5])
0.6798503398895264
Train performance at epoch 4 is 0.9250
torch.Size([40, 5])
1.88315749168396
Train performance at epoch 5 is 0.9390
torch.Size([40, 5])
5.0895538330078125
Train performance at epoch 6 is 0.9430
torch.Size([40, 5])
0.47974300384521484
Train performance at epoch 7 is 0.9490
torch.Size([40, 5])
0.07137546688318253
Train performance at epoch 8 is 0.9670
torch.Size([40, 5])
0.04734325408935547
Train performance at epoch 9 is 0.9850
torch.Size([40, 5])
0.2498922348022461
Train performance at epoch 10 is 0.9830
torch.Size([40, 5])
0.0
Train performance at epoch 11 is 0.9860


In [50]:
class_correct = list(0. for i in range(5))
class_total = list(0. for i in range(5))
classes = ['airplane', 'campfire', 'key', 'moon', 'palm tree']
mini_batch_size = 64
num_batches = int(len(X_dev)/ (4 * mini_batch_size))
num_remaining = len(X_dev) - num_batches * mini_batch_size

with torch.no_grad():
    for t in range(num_batches):
        rand_indices = np.random.choice(len(X_dev), mini_batch_size)
        sketch = torch.from_numpy(X_dev[rand_indices, :, :, :])
        labels = torch.from_numpy(Y_dev[rand_indices])

#     for i in range(len(X_dev)):
#         sketch = torch.from_numpy(X_dev[i])
#         label = Y_dev[i]
        print('starting batch: ', t)
        outputs = bestModel.forward(sketch)
        _, predictions = torch.max(outputs, 1)
        for i in range(mini_batch_size):
            predicted = predictions[i].item()
            label = labels[i].item()
        
            class_total[int(label)] += 1
            if predicted == label:
                class_correct[predicted] += 1


for i in range(5):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

starting batch:  0
starting batch:  1
starting batch:  2
starting batch:  3
starting batch:  4
starting batch:  5
starting batch:  6
starting batch:  7
starting batch:  8
starting batch:  9
starting batch:  10
starting batch:  11
starting batch:  12
starting batch:  13
starting batch:  14
starting batch:  15
starting batch:  16
starting batch:  17
starting batch:  18
starting batch:  19
starting batch:  20
starting batch:  21
starting batch:  22
starting batch:  23
starting batch:  24
starting batch:  25
starting batch:  26
starting batch:  27
starting batch:  28
starting batch:  29
starting batch:  30
starting batch:  31
starting batch:  32
starting batch:  33
starting batch:  34
starting batch:  35
starting batch:  36
starting batch:  37
starting batch:  38
starting batch:  39
starting batch:  40
starting batch:  41
starting batch:  42
starting batch:  43
starting batch:  44
starting batch:  45
starting batch:  46
starting batch:  47
starting batch:  48
starting batch:  49
starting b

starting batch:  396
starting batch:  397
starting batch:  398
starting batch:  399
starting batch:  400
starting batch:  401
starting batch:  402
starting batch:  403
starting batch:  404
starting batch:  405
starting batch:  406
starting batch:  407
starting batch:  408
starting batch:  409
starting batch:  410
starting batch:  411
starting batch:  412
starting batch:  413
starting batch:  414
starting batch:  415
starting batch:  416
starting batch:  417
starting batch:  418
starting batch:  419
starting batch:  420
starting batch:  421
starting batch:  422
starting batch:  423
starting batch:  424
starting batch:  425
starting batch:  426
starting batch:  427
starting batch:  428
starting batch:  429
starting batch:  430
starting batch:  431
starting batch:  432
starting batch:  433
starting batch:  434
starting batch:  435
starting batch:  436
starting batch:  437
starting batch:  438
starting batch:  439
starting batch:  440
starting batch:  441
starting batch:  442
starting batc